## LinuxDay 2023

#### Esempio di codice difficile da leggere

Esempio:
Prendo una ricetta per una pietanza ed aggiusto le quantità degli ingredienti adattandole al nuovo numero di porzioni da realizzare.
Il primo elemento di una ricetta è il numero di porzioni per cui è riportata, mentre i rimanenti sono delle tuple con indicato il nome dell'ingrediente, la quantità e l'unità di misura in cui è espressa:
(nome, quantità, unità)

In [1]:
# take a meal recipe and change the number of servings by adjusting the number of servings
# A recipe's first element is the number of servings, and the remainder of element is (name, amount, unit), 
# such as ("flour", 1.5,"cup")
def adjust_recipe(recipe, servings):
    new_recipe = [servings]
    old_servings = recipe[0]
    factor = servings / old_servings
    recipe.pop(0)
    while recipe:
        ingredient, amount, unit = recipe.pop(0)
        # please only use numbers that will be easily measurable
        new_recipe.append((ingredient, amount * factor, unit))
    return new_recipe

In [2]:
adjust_recipe([2,("pane",2,"fetta"),("olio",1,"cucchiaio")],3)

[3, ('pane', 3.0, 'fetta'), ('olio', 1.5, 'cucchiaio')]

Posso provare una nuova versione, variando la struttura di dati sottostante:

In [3]:
# VERSIONE ALTERNATIVA
def adjust_recipe2(recipe, servings):
    old_servings = recipe.pop(0)
    factor = servings / old_servings
    new_recipe = {ingredient: (amount * factor, unit) for ingredient, amount, unit in recipe}
    new_recipe["servimgs"] = servings
    return new_recipe

In [4]:
adjust_recipe2([2,("pane",2,"fetta"),("olio",1,"cucchiaio")],3)

{'pane': (3.0, 'fetta'), 'olio': (1.5, 'cucchiaio'), 'servimgs': 3}

Il codice, sebbene più compatto, non è migliorato per la sua comprensibilità.

#### L'annotazione del codice fornisce una maggiore chiarezza sugli intenti del programmatore, aumentando la comprensibilità del codice.

In [27]:
import typing
import datetime
from dataclasses import dataclass
from enum import auto, Enum

class ImperialMeasure(Enum):
    """
    Unità di misura utilizzate per definire gli ingredienti
    """
    TEASPOON = auto()
    TABLESPOON = auto()
    CUP = auto()
        
class Broth(Enum):
    """
    Tipologie di brodo disponibili
    """
    VEGETABLE = auto()
    CHICKEN = auto()
    BEEF = auto()
    FISH = auto()

@dataclass(frozen=True)
class Ingredient:
    """
    Definizione del generico ingrediente
    @param name:   stringa che descrive il nome dell'ingrediente
    @param amount: quantità di ingrediente presente nella ricetta
    @param units:  unità di misura relativa alla quantità di ingrediente
    """
    name: str
    amount: float = 1
    units: ImperialMeasure = ImperialMeasure.CUP

@dataclass
class Recipe:
    aromatics: set[Ingredient]
    broth: Broth
    vegetables: set[Ingredient]
    meats: set[Ingredient]
    starches: set[Ingredient]
    garnishes: set[Ingredient]
    time_to_cook: datetime.timedelta
    def make_vegetarian(self)->None:
        self.meats.clear()
        self.broth = Broth.VEGETABLE
    def get_ingredient_names(self)->set[str]:
        ingredients = (self.aromatics |
                       self.vegetables |
                       self.meats |
                       self.starches |
                       self.garnishes
                      )
        return ({i.name for i in ingredients} | {self.broth.name.capitalize() + " broth "})


Definizione degli ingrqedienti:

In [28]:
pepper = Ingredient("Pepper", 1, ImperialMeasure.TABLESPOON)
garlic = Ingredient("Garlic", 2, ImperialMeasure.TEASPOON)
carrots = Ingredient("Carrot", .25, ImperialMeasure.CUP)
celery = Ingredient("Celery", .25, ImperialMeasure.CUP)
onions = Ingredient("Onions", .25, ImperialMeasure.CUP)
parsley = Ingredient("Parsley", 2, ImperialMeasure.TABLESPOON)
noodles = Ingredient("Noodles", 1.5, ImperialMeasure.CUP)
chicken = Ingredient("Chicken", 1.5, ImperialMeasure.CUP)

Definizione della ricetta:

In [29]:
chicken_noodles_soup = Recipe(
    aromatics = {pepper, garlic},
    broth = Broth.CHICKEN,
    vegetables = {celery, onions, carrots},
    meats = {chicken},
    starches = {noodles},
    garnishes = {parsley},
    time_to_cook = datetime.timedelta(minutes = 60)
)

In [31]:
chicken_noodles_soup.get_ingredient_names()

{'Carrot',
 'Celery',
 'Chicken',
 'Chicken broth ',
 'Garlic',
 'Noodles',
 'Onions',
 'Parsley',
 'Pepper'}

#### Esempio sull'utilizzo dei protocolli (Protocols)

In [66]:
from typing import Protocol
from dataclasses import dataclass

class Movable(Protocol):
    x: float
    y: float
    def move(a:float, b:float)->Movable:
        """Metodo non implementato"""

class Rectangle():
    def __init__(self, 
                 x: float, 
                 y: float,
                 l: float, 
                 h: float):
            self.x = x
            self.y = y
            self.l = l
            self.h = h
    def __str__(self):
        return f"Rectangle: ({self.x},{self.y},{self.l},{self.h})"
    def move(self, deltax: float, deltay: float)->Rectangle:
        self.x += deltax
        self.y += deltay
        return self

class Circle():
    def __init__(self, 
                 x: float, 
                 y: float,
                 r: float):
            self.x = x
            self.y = y
            self.r = r
    def __str__(self):
        return f"Circle: ({self.x},{self.y},{self.r})"
    def move(self, deltax: float, deltay: float)->Circle:
        self.x += deltax
        self.y += deltay
        return self

#Definisco una funzione "generica" che funziona quando l'oggetto è muovibile
def move_list(lista: list[Movable], x:float = 0, y:float = 0)->list[Movable]:
    for oggetto in lista:
        if not(oggetto is None):
            oggetto.move(x,y)
    return lista

In [73]:
R1 = Rectangle(0,0,10,5)
C1 = Circle(1,1,3)

print(R1)

Rectangle: (0,0,10,5)


In [74]:
ML1 = [C1,R1]

ML2 = move_list(ML1,1,1)
for x in ML2:
    print(x)

Circle: (2,2,3)
Rectangle: (1,1,10,5)


#### Esempi di Pattern Matching

In [16]:
def scegli(x: int)->int:
    match x:
        case 1:
            return 10
        case 2:
            return 20
        case 3:
            return 40
        case _:
            return 0

In [18]:
scegli((2,2))

0

In [23]:
def scegli2(x: tuple[int,int])->int:
    match x:
        case 1, 1:
            return 10
        case _, 1:
            return 20
        case 1, _:
            return 40
        case _:
            return 0

In [24]:
scegli2((1,2))

40

In [21]:
def scegli3(x: dict)->int:
    match x:
        case {"tipo": "numero"}:
            return 1
        case {"tipo": "figura", "forma": "Cerchio"}:
            return 2
        case {"tipo": "figura", "forma": "Rombo"}:
            return 3
        case _:
            return 0

In [30]:
scegli3({"nome":"Giuseppe", "tipo": "numero"})

1